In [1]:
import os
import dotenv

dotenv.load_dotenv()
try:
    os.environ["OPENAI_API_KEY"]
except KeyError:
    raise Exception("OPENAI_API_KEY is not set in your .env file")

In [2]:
import sqlite3

# Connect to the SQLite database and load descriptions
conn = sqlite3.connect('videos.db')
cursor = conn.cursor()
# Execute the SQL query to fetch descriptions and categories
cursor.execute("SELECT Title, Chapter FROM VideoChapters")
rows = cursor.fetchall()
# Extract descriptions and categories from the query result
titles = [row[0] for row in rows]
chapters = [row[1] for row in rows]
# Close the connection
conn.close()

In [3]:
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

    def __repr__(self):
        return f"Document(page_content='{self.page_content}', metadata={self.metadata})"

In [4]:
# Convert rows to Document objects
docs = []
for title, chapter in rows:
    # Determine if the chapter is present
    has_chapter = chapter is not None
    
    # Create a Document object for each row
    doc = Document(
        page_content=f"Title: {title}, Chapter: {chapter if has_chapter else 'No chapter'}",
        metadata={"has_chapter": has_chapter}
    )
    docs.append(doc)

print(docs)

[Document(page_content='Title: أسرع وأسهل طريقة لضرب أي عدد في 11؟ - الحساب الذهني, Chapter: No chapter', metadata={'has_chapter': False}), Document(page_content='Title: هل يمكنك جعل ثلاثة أصفار تساوي 6؟ - حل لغز العدد 6 للأذكياء, Chapter: No chapter', metadata={'has_chapter': False}), Document(page_content='Title: كم تبلغ مساحة القطاع الأزرق داخل هذا القرص؟ - تمارين وحلول, Chapter: No chapter', metadata={'has_chapter': False}), Document(page_content='Title: الدوال كمفهوم - الدالة ومكوناتها - الجزء الأول, Chapter: No chapter', metadata={'has_chapter': False}), Document(page_content='Title: الدوال كمفهوم - الدالة ومكوناتها - الجزء الأول, Chapter: No chapter', metadata={'has_chapter': False}), Document(page_content='Title: الدوال كمفهوم - قصور الدالة - الجزء الثاني, Chapter: No chapter', metadata={'has_chapter': False}), Document(page_content='Title: الدوال كمفهوم - الغير معرف - الجزء الثالث, Chapter: No chapter', metadata={'has_chapter': False}), Document(page_content='Title: الحلقة 1: 

In [6]:
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings


db = Chroma.from_documents(docs, OpenAIEmbeddings())

In [23]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_openai import ChatOpenAI


metadata_field_info = [
    AttributeInfo(
        name="has_chapter",
        description="Indicates whether the video title contains chapters. If 'True', chapters are present, which detail the main topics or sections of the video. If 'False', the video does not contain any chapters.",
        type="boolean",
    )
]

document_content_description = "Titles of videos with their corresponding chapters."

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

first_retriever = SelfQueryRetriever.from_llm(
    llm,
    db,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
    search_kwargs={"k": 1},
)

In [33]:
# Combine context and question into a single query string
query = "فيديو يتحدث عن البديهيات"

In [42]:
first_result = first_retriever.invoke({"query": query})
first_result

[Document(page_content='Title: الحلقة 1: مقدمة إلى البرهان الرياضياتي, Chapter: شرح البديهيات', metadata={'has_chapter': True})]

In [30]:
# from langchain.prompts import PromptTemplate

# # Define your custom prompt template for retrieving video data
# custom_prompt_template = PromptTemplate.from_template(
#     """Given the following question, return the exact video title in Arabic and relevant metadata (chapter).
#     Do a keyword search first then a semantic search.
#     If more than one document has been used, separate them with ONLY new line \n.
#     You should at least retrieve ONE document.
#     If you didn't find any relevant document, retrieve the closest one to the question, based on chapters and content.
#     Don't retrieve the same video document twice.
#     Context: 
#     {context}
#     Each document in the context is presented with Title followed by its chapter in the metadata.
#     Question: {question}
#     Answer:"""
# )

# # Assuming 'vectorstore' is properly set up for videos
# second_retriever = db.as_retriever(
#     search_type="mmr", search_kwargs={
#         "fetch_k": 10, 
#         "lambda_mult": 0.8
#     }
# )

# # Modify the format_docs function to accept the documents list
# def format_docs(docs):
#     formatted_docs = []
#     for i, doc in enumerate(docs, start=1):
#         doc_description = f"Document {i}:\nTitle: {doc.page_content}"
#         doc_chapter = f"Chapter: {doc.metadata.get('chapter', 'N/A')}"  # Use 'N/A' if 'chapter' is not found
#         formatted_doc = f"{doc_description}\n{doc_chapter}\n-----"
#         formatted_docs.append(formatted_doc)
#     return "\n".join(formatted_docs)


In [34]:
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser

# rag_chain = (
#     {"context": second_retriever | format_docs, "question": RunnablePassthrough()}
#     | custom_prompt_template
#     | llm
#     | StrOutputParser()
# )

# output = rag_chain.invoke(query)
# print(output)

Title: الحلقة 1: مقدمة إلى البرهان الرياضياتي, Chapter: شرح البديهيات


In [36]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain

db = SQLDatabase.from_uri("sqlite:///videos.db")
print(db.dialect)
print(db.get_usable_table_names())
# Connect to the SQLite database for SQL query execution
sql_chain = create_sql_query_chain(llm, db)

sqlite
['VideoChapters']


In [43]:
import ast

# Function to extract title and chapter from the page_content
def extract_title_and_chapter(page_content):
    # Assuming the format is 'Title: <title>, Chapter: <chapter>'
    title_part, chapter_part = page_content.split(", Chapter: ")
    title = title_part.split("Title: ")[1]
    chapter = chapter_part
    return title, chapter

sql_outputs = []
for document in first_result:
    title, chapter = extract_title_and_chapter(document.page_content)
    # Dynamically construct SQL query
    sql_query = f"SELECT Title, Link FROM VideoChapters WHERE Title = '{title}' AND Chapter = '{chapter}'"
    sql_output = db.run(sql_query)
    print(sql_query)
    try:
        parsed_output = ast.literal_eval(sql_output)
        if isinstance(parsed_output, list):
            for record in parsed_output:
                # Assuming record format is correct and contains all needed fields
                merged_answer = f"{record[0]} {record[1]}"
                sql_outputs.append(merged_answer)
        else:
            print(f"Unexpected format: {sql_output}")
    except (ValueError, SyntaxError):
        print(f"Could not parse output: {sql_output}")
    print(sql_output)

SELECT Title, Link FROM VideoChapters WHERE Title = 'الحلقة 1: مقدمة إلى البرهان الرياضياتي' AND Chapter = 'شرح البديهيات'
[('الحلقة 1: مقدمة إلى البرهان الرياضياتي', 'https://youtu.be/3ZsosFpV410&t=160')]
